In [1]:
import sys
sys.path.insert(0, '../../')
from time import sleep

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyplr import stlab

In [2]:
d = stlab.STLAB(username='admin', identity=1, password='83e47941d9e930f6')

STLAB device setup complete...


In [20]:
def interp_spectra(spectra):
    '''
    This function needs generalising.

    Parameters
    ----------
    spectra : TYPE
        DESCRIPTION.

    Returns
    -------
    intp_tbl : TYPE
        DESCRIPTION.

    '''
    #tbl = spectra.unstack(level=0)
    #tbl.columns = [val[1] for val in tbl.columns]
    
    intp_tbl = pd.DataFrame()
    for led, df in spectra.groupby(['led']):
        intensities = df.index.get_level_values('intensity')
        new_intensities = np.linspace(intensities.min(), intensities.max(), 4096)
        new_intensities = new_intensities.astype('int')
        df.reset_index(inplace=True, drop=True)
        #df.columns = range(0, df.shape[1])
        df.index = df.index * 63
        n = df.reindex(new_intensities).interpolate(method='linear')
        n['intensity'] = n.index
        n['led'] = led
        intp_tbl = intp_tbl.append(n)
    intp_tbl.set_index(['led','intensity'], inplace=True)
    return intp_tbl

In [5]:
lkp = pd.read_csv('/Users/jtm/Code/cvd_pupillometry/PyPlr/pyplr/data/corrected_oo_lookup_table.csv', index_col=['led','intensity'])
LED3_max = 1800 # comfortable maximal melanopsin stim
r_matches = {}
for led in range(10):
    match = stlab.led_radiance_match(3, LED3_max, led, lkp)
    r_matches[led] = match

r_matches

{0: (0.004238363945272994, 2127),
 1: (0.0005316173873932684, 1692),
 2: (0.0028045785748886942, 1656),
 3: (0.0, 1800),
 4: (0.0008152973939665742, 1537),
 5: (0.0001710459224302241, 2113),
 6: (0.006184807102723511, 695),
 7: (0.006900740582779008, 831),
 8: (8.106384311723502e-05, 2795),
 9: (0.000973117048275185, 1431)}

In [4]:
d.set_spectrum_a([0,0,0,2000,0,0,0,0,0,0])

In [6]:
from scipy.signal import chirp

for led in r_matches.keys():

    # stimulus params
    videoname = 'chirp_{}'.format(led)
    Fs = 100
    mintensity = 0
    maxtensity = r_matches[led][1]

    # stimulus params
    background  = maxtensity/2
    amplitude   = maxtensity/2
    duration = 20
    n = 2000
    t = np.linspace(0, duration, n, endpoint=False)
    f0 = 0.01
    f1 = 1

    # define frequency modulation
    chp = chirp(t, f0, duration, f1, method='linear', phi=270)
    chp = stlab.modulate_intensity_amplitude(chp, background, amplitude)
    #fig, ax = plt.subplots(figsize=(12,4))
    #ax.plot(chp)
    #ax.set_ylabel("Intensity")
    #ax.set_xlabel("Time")

    # define amplitude modulation
    time  = stlab.get_time_vector(duration)
    sm = stlab.sinusoid_modulation(.5, duration, Fs)
    ivals = stlab.modulate_intensity_amplitude(sm, background, np.linspace(0,amplitude,duration*Fs))
    #fig, ax = plt.subplots(figsize=(12,4))
    #ax.plot(ivals)
    #ax.set_ylabel("Intensity")
    #ax.set_xlabel("Time");

    duration=60
    time = stlab.get_time_vector(duration)
    stim = np.zeros(duration*Fs)

    # assemble time course
    stim[0:100]     = maxtensity # 1s pulse
    stim[800:900]   = background # step
    stim[900:2900]  = chp  # chirp
    stim[2900:3100] = background
    stim[3100:5100] = ivals
    stim[5100:5300] = background
    stim = stim.astype('int')

    # plot
    #fig, ax = plt.subplots(figsize=(16,4))
    #ax.plot(stim)
    #ax.set_ylabel("Intensity")
    #ax.set_xlabel("Time")

    # make video file
    cols = stlab.get_video_cols()
    zs = np.zeros(duration*Fs)
    zs = zs.astype('int')
    data = [zs for x in range(10)]
    data[led] = stim

    #data = [stim for x in range(10)]
    data.insert(0, time)
    df = pd.DataFrame(data, cols).T
    stlab.make_video_file(df, videoname)

"chirp_0.dsf" saved in the current working directory.
"chirp_1.dsf" saved in the current working directory.
"chirp_2.dsf" saved in the current working directory.
"chirp_3.dsf" saved in the current working directory.
"chirp_4.dsf" saved in the current working directory.
"chirp_5.dsf" saved in the current working directory.
"chirp_6.dsf" saved in the current working directory.
"chirp_7.dsf" saved in the current working directory.
"chirp_8.dsf" saved in the current working directory.
"chirp_9.dsf" saved in the current working directory.


In [8]:
import glob
from time import sleep

for chp in glob.glob('chirp_*.dsf'):
    d.load_video_file(chp)
    d.play_video_file()
    sleep(60)

video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...


In [28]:
glob.glob('chirp_*.dsf')

['chirp_4.dsf',
 'chirp_5.dsf',
 'chirp_7.dsf',
 'chirp_6.dsf',
 'chirp_2.dsf',
 'chirp_3.dsf',
 'chirp_1.dsf',
 'chirp_0.dsf',
 'chirp_8.dsf',
 'chirp_9.dsf']